# **Flappy Bird : Test de différents paramètres d'apprentissage**

Les tests ont été réalisés pour l'environnement Flappy Bird (https://github.com/davHub/flappy-bird-env.git) dans le cas où l'observation est l'image entière $\Rightarrow$ flappyBird_cnn

## Détail de l'environnement
Description FlappyBird_env: 
- 1 bird 
- Observation : image 84 x 84 x 1 (grayscale from -1 to 1)
- Reward : +1 à chaque step, 0 si done

## Paramétrage standard
Quand rien n'est spécifié:

- Architecture: 8x8_Conv2D(32) - ReLU - 4x4_Conv2D(64) - ReLU - 3x3_Conv2D(64) - ReLU - Flatten - Dense(512) - ReLU - Dense(2)
- Gamma : $\gamma=0.975$

### **Sommaire**
1. [Test 1 : Best model ~60 points en moyenne](#test-1)<br>
    1.1 : [Re-entraînement : Best model ~80 points en moyenne](#retrain1)<br>
    
2. [Test 2 : Best model ~12 points en moyenne](#test-2)<br>

3. [Comparaison](#comp)<br>
    3.1 [Comparaison Test 1 vs Test 2](#comp1vs2)<br>


<a id="test-1"></a>
## **1. Test 1 : ~ 60 points**

### Paramétrage global

> $\alpha$ = 0.00025 | $\gamma$ = 0.975  |  $\epsilon_{deb}$=1.0  |   $\epsilon_{fin}$= 0.1  |  $batch\_size$ = 32   |   $memory\_size$ = 10 000    |    $C$ = 200  |  $num\_steps$ = 2 000 000
 Experience Replay : Simple Buffer


### Résultats

Durée apprentissage : <em style='color:red'> 6h23 </em>

![alt text](plot_img/Qvalue-full-image1.gif "Q-Value")

### Interprétation/Conclusion

<a id="retrain1"></a>
### **1.1 Retrain : ~ 80 points**

Re-apprentissage à partir du dernier modèle enregistré (step 2M).

### Paramétrage global

> $\alpha$ = 0.00025 | $\gamma$ = 0.975  |  $\epsilon_{deb}$=0.1  |   $\epsilon_{fin}$= 0.01  |  $batch\_size$ = 32   |   $memory\_size$ = 10 000    |    $C$ = 200  |  $num\_steps$ = 1 000 000

### Résultats

Durée apprentissage : <em style='color:red'> 3h24 </em>

![alt text](plot_img/Qvalue-full-image2.gif "Q-Value")
![alt text](plot_img/Qvalue-full-image2.gif "Q-Value")


**GradCAM & ExtractActivations**

![alt_text](plot_img/tf_explain/Flappy_Test_1/extractActiv_test1-2M.gif "Extract Activation 2 millions")
![alt_text](plot_img/tf_explain/Flappy_Test_1/gradCam_test1-2M.gif "GradCAM 2 millions")
### Interprétation/Conclusion

De très bons résultats. Le paramètre $\gamma$ a été choisi de tel sorte qu'il que Flappy voit plus loin que la prochaine plateforme (entre 2 et 3 plateformes)

<a id="test-2"></a>
## **2. Tests 2 : ~10 points**

### Paramétrage global

> $\alpha$ = 0.001 | $\gamma$ = 0.975   |  $\epsilon_{deb}$=1.0  |   $\epsilon_{fin}$= 0.1  |  $batch\_size$ = 32   |   $memory\_size$ = 50 000    |    $C$ = 10 000  |   $num\_steps$ = 6 000 000
\\ Experience Replay : Prioritized ($\alpha = 0.6,\ \ \beta=0.4\ \ \epsilon=1.e-6$)

### Résultats

Durée apprentissage : <em style='color:red'> 23h56 </em>

![alt text](plot_img/Value-full-image3.gif "Value")
![alt text](plot_img/Qvalue-full-image3.gif "Q-Value")


*Explication de la première couche :*

**Grad CAM** de la 1ère couche après 200 000, 3M et 6M de steps :

![alt_text](plot_img/tf_explain/Flappy_Test_2/gradCam_test2-200m.gif "GradCAM 200 000")
![alt_text](plot_img/tf_explain/Flappy_Test_2/gradCam_test2-3M.gif "GradCAM 3 millions")
![alt_text](plot_img/tf_explain/Flappy_Test_2/gradCam_test2-6M.gif "GradCAM 6 millions")

**Activation extraction** de la 1ère couche après 200 000, 3M et 6M de steps :

![alt_text](plot_img/tf_explain/Flappy_Test_2/extractActiv_test2-200m.gif "Extract Activation 200 000")
![alt_text](plot_img/tf_explain/Flappy_Test_2/extractActiv_test2-3M.gif "Extract Activation 3 millions")
![alt_text](plot_img/tf_explain/Flappy_Test_2/extractActiv_test2-6M.gif "Extract Activation 6 millions")

Ici, on observe ce que chacun des 32 filtres de la première couche filtre. Le résultat après 3M de steps est plutôt bien mais pas celui après 6M de steps. En effet seulement quelques filtres semblent donner une quelconque information. Cela est peut-être du à une sorte d'over-fitting (Il faudrait réessayer en ajoutant du Dropout).

### Interprétation/Conclusion
On remarque une amélioration jusqu'à un peu plus de la moitié de l'apprentissage. Notamment les résultats du réseau en ce qui concerne la localisation des zones d'importances de l'image (gradCAM). Après un certain moment cependant, les résultats deviennent de moins en moins bons.
Tout de même, les résultats de cet apprentissage sont très médiocre. Deux possibilités peuvent en être la cause : le prioritized replay ou le paramètre C qui est trop grand.

<a id="comp"></a>
##  **3. Comparaison**

Comparaison des apprentissages de plusieurs tests.

<a id="comp1vs2"></a>
###  **3.1 Comparaison Test 1 vs Test 2**
![alt text](plot_img/vs/1vs2-eps-value.png "Value Comparison")
![alt text](plot_img/vs/1vs2-eps-rew.png "Reward Comparison")
![alt text](plot_img/vs/1vs2-eps-loss.png "Loss Comparison")

### Interprétation/Conclusion

Pour la fonction de valeur, le test 1 va bien plus rapidement vers une valeur très proche de la valeur espérée. Il produit également de meilleurs résultats pour une raison que je n'esplique pas encore (peut-être le paramètre C).